In [2]:
import os
import pandas
import numpy
import utils
import tensorflow as tf
import tflearn
from amino_acid import Data

curr_dir = os.path.dirname(os.getcwd())
path_to_data = curr_dir + '/mhcPreds/data/'
tensorboard_path = curr_dir + '/mhcPreds/tensorboard/'
models_path = curr_dir + '/mhcPreds/models/'


train_dat = Data(path_to_data + 'train.txt', allele='HLA-A-0101')
test_dat = Data(path_to_data + 'test.txt', allele='HLA-A*01:01')


onehot, aff_one_hot, idx_one_hot = train_dat.one_hot_encoding(kmer_size=9)
test_onehot, test_aff_one_hot, test_idx_one_hot = test_dat.one_hot_encoding(kmer_size=9)


"""
kmer, aff_kmer, idx_kmer = train_dat.kmer_index_encoding(kmer_size=9)
aff_kmer = utils.ic50_to_regression_target(aff_kmer, max_ic50=50000)
aff_one_hot = utils.ic50_to_regression_target(aff_one_hot, max_ic50=50000)
"""

xTr, xTe, yTr, yTe = onehot, test_onehot, aff_one_hot, test_aff_one_hot
yTr = numpy.reshape(yTr, (yTr.shape[0], 1))
yTe = numpy.reshape(yTe, (yTe.shape[0], 1))
xTr = numpy.reshape(xTr, (xTr.shape[0], xTr.shape[1], xTr.shape[2], 1))
xTe = numpy.reshape(xTe, (xTe.shape[0], xTe.shape[1], xTe.shape[2], 1))
    

/Users/carlomazzaferro/anaconda/envs/MHC_NN/lib/python3.5/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/carlomazzaferro/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


In [7]:
fs = xTr[0, :, :]

In [14]:
fsv

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 

In [25]:
fsv = numpy.reshape(fs, (20,9))
idx = numpy.array(list(zip(numpy.nonzero(fsv)[0], numpy.nonzero(fsv)[1]))).astype('int')
idx

array([[ 0,  0],
       [ 2,  2],
       [ 6,  0],
       [ 7,  2],
       [10,  2],
       [11,  6],
       [14,  7],
       [16,  6],
       [18,  7]])

In [23]:
values = numpy.ones(9).astype(int)
values

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [24]:
sparse = tf.SparseTensor(idx, values, dense_shape=[9,20])

In [26]:

batch_size = 75
epochs = 1500
lr = 0.00001

network = tflearn.input_data(shape=[None, 9, 20, 1])
network = tflearn.dropout(network, 0.4)
print(type(network.))
loss = tflearn.fully_connected(network, 1, activation='sigmoid')
network = tflearn.regression(loss, optimizer='momentum', learning_rate=0.001)
model = tflearn.DNN(network, checkpoint_path=tensorboard_path, max_checkpoints=1, tensorboard_verbose=3)
model.fit(xTr, yTr, n_epoch=1000, validation_set=0.1, shuffle=True,
          show_metric=True, batch_size=64)

<class 'tensorflow.python.framework.ops.Tensor'>


IndexError: list index out of range

In [106]:
yTe_scoring

array([ 0.32163885,  0.32163885,  0.32163885, ...,  0.33125994,
        0.08468665,  0.08468665])

In [8]:
import pandas
preds = [regression_target_to_ic50(i[0]) for i in preds]
targs = [regression_target_to_ic50(i[0]) for i in yTe]
pandas.DataFrame([preds, targs]).T

,0,1
0,322.757333,1540.276452
1,298.288057,1540.276452
2,281.972457,1540.276452
3,272.610725,1540.276452
4,396.318246,1540.276452
5,330.758850,1540.276452
6,358.977761,1540.276452
7,266.020782,1540.276452
8,246.968689,1540.276452
9,3648.759284,1540.276452


In [11]:
numpy.max(xTe)

20

In [5]:
import pandas
ps = model.predict(xTe)
preds = [regression_target_to_ic50(i[0]) for i in ps]
targs = [regression_target_to_ic50(i[0]) for i in yTe]
pandas.DataFrame([preds, targs]).T


,0,1
0,13779.433073,20000.000000
1,9219.578326,50000.000000
2,986.525863,3106.005166
3,1236.310011,50000.000000
4,3050.660671,20000.000000
5,1948.502773,20000.000000
6,11.198050,33.000000
7,41.551778,1194.731086
8,4758.773243,13633.749813
9,29.460669,22.629947
